<h2 style="text-align: center;">Data Mining & Grundlagen Maschinelles Lernen 1</h2>
<h3 style="text-align: center;">Wintersemester 2023/24</h3>
<h4 style="text-align: center;">Semesterprojekt: Umsatzvorhersage</h4>


# Business Understanding

Im Rahmen des Semesterprojekts für das Modul "Data Mining & Grundlagen Maschinelles Lernen 1" widmen wir uns einer praxisnahen Aufgabe – der Umsatzvorhersage für eine große Supermarktkette in Deutschland. Unser Ziel ist es, ein zuverlässiges Vorhersagemodell zu entwickeln, das auf Basis historischer Daten und verschiedener Einflussfaktoren den erwarteten Tagesumsatz jeder Filiale prognostiziert.
Eine genaue Vorhersage ermöglicht es den Filialleitungen, die Anzahl der Mitarbeiterinnen und Mitarbeiter optimal zu planen und so die Kundenzufriedenheit zu maximieren, während gleichzeitig die Personalkosten minimiert werden.

# Data Understanding

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from math import *
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, accuracy_score, r2_score
from sklearn.tree import DecisionTreeRegressor
import time
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit, cross_val_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier
from sklearn.inspection import permutation_importance
import plotly.express as px
import matplotlib as plt
import seaborn as sns

In [4]:
# Schritt 1: Laden der Datensätze
train_df = pd.read_csv('../data/raw/dmml1_train.csv')
store = pd.read_csv('../data/raw/dmml1_stores.csv')

In [5]:
train_df.describe()

,Store ID,DayOfWeek,Sales,Customers,Open,Promo,SchoolHoliday
count,246903.000000,246903.000000,246903.000000,246903.000000,246903.000000,246903.000000,246903.000000
mean,150.568879,3.996898,5844.491894,639.519524,0.830354,0.380490,0.178763
std,86.651448,1.998008,4007.013717,472.495279,0.375322,0.485508,0.383154
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,76.000000,2.000000,3745.000000,405.000000,1.000000,0.000000,0.000000
50%,150.000000,4.000000,5719.000000,617.000000,1.000000,0.000000,0.000000
75%,226.000000,6.000000,7884.000000,833.000000,1.000000,1.000000,0.000000
max,300.000000,7.000000,38025.000000,7388.000000,1.000000,1.000000,1.000000


In [6]:
store.describe()

,Store ID,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear
count,300.000000,298.000000,190.000000,190.000000,300.000000,141.000000,141.000000
mean,150.500000,5368.590604,7.189474,2007.794737,0.470000,22.886525,2011.978723
std,86.746758,7694.152795,3.236023,9.329079,0.499933,14.235816,1.627742
min,1.000000,30.000000,1.000000,1900.000000,0.000000,1.000000,2009.000000
25%,75.750000,682.500000,4.000000,2006.000000,0.000000,10.000000,2011.000000
50%,150.500000,2270.000000,8.000000,2009.000000,0.000000,22.000000,2012.000000
75%,225.250000,7107.500000,10.000000,2012.000000,1.000000,37.000000,2013.000000
max,300.000000,58260.000000,12.000000,2015.000000,1.000000,50.000000,2015.000000


In [7]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246903 entries, 0 to 246902
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Store ID       246903 non-null  int64 
 1   DayOfWeek      246903 non-null  int64 
 2   Date           246903 non-null  object
 3   Sales          246903 non-null  int64 
 4   Customers      246903 non-null  int64 
 5   Open           246903 non-null  int64 
 6   Promo          246903 non-null  int64 
 7   StateHoliday   246903 non-null  object
 8   SchoolHoliday  246903 non-null  int64 
dtypes: int64(7), object(2)
memory usage: 17.0+ MB


In [8]:
store.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Store ID                   300 non-null    int64  
 1   StoreType                  300 non-null    object 
 2   Assortment                 300 non-null    object 
 3   CompetitionDistance        298 non-null    float64
 4   CompetitionOpenSinceMonth  190 non-null    float64
 5   CompetitionOpenSinceYear   190 non-null    float64
 6   Promo2                     300 non-null    int64  
 7   Promo2SinceWeek            141 non-null    float64
 8   Promo2SinceYear            141 non-null    float64
 9   PromoInterval              141 non-null    object 
dtypes: float64(5), int64(2), object(3)
memory usage: 23.6+ KB


In [9]:
train_df.isna().sum()

Store ID         0
DayOfWeek        0
Date             0
Sales            0
Customers        0
Open             0
Promo            0
StateHoliday     0
SchoolHoliday    0
dtype: int64

In [10]:
store.isna().sum()

Store ID                       0
StoreType                      0
Assortment                     0
CompetitionDistance            2
CompetitionOpenSinceMonth    110
CompetitionOpenSinceYear     110
Promo2                         0
Promo2SinceWeek              159
Promo2SinceYear              159
PromoInterval                159
dtype: int64

# Data Preparation (Aufgabe 2)

In [11]:
train_df = pd.read_csv('../data/raw/dmml1_train.csv')
store = pd.read_csv('../data/raw/dmml1_stores.csv')

In [12]:
# Tausche Codierung für bessere Lesbarkeit
train_df['StateHoliday'] = train_df['StateHoliday'].map({'0': "Kein Feiertag", 'a': "Feiertag", 'b': "Ostern", 'c': "Weihnachten"})
store['StoreType'] = store['StoreType'].map({'a': "StoreType A", 'b': "StoreTypee B", 'c': "StoreType C", 'd': "StoreType D"})
store['Assortment'] = store['Assortment'].map({'a': "Basic", 'b': "Extra", 'c': "Extended"})

In [13]:
# Fehlende Werte in "CompetitionDistance" werden durch den Mittelwert ersetzt
store["CompetitionDistance"].replace(np.nan, store["CompetitionDistance"].mean(), inplace=True)
# Neue Spalten für den Tag, den Monat, das Jahr und Kalenderwoche werdne aus dem Datum extrahiert
# Date wird  auch in ein DateTime Objekt umgewandelt

train_df['Date'] = pd.to_datetime(train_df['Date'])
train_df['Year'] = train_df['Date'].dt.year
train_df['Month'] = train_df['Date'].dt.month
train_df['Day'] = train_df['Date'].dt.day
train_df['WeekOfYear'] = train_df['Date'].dt.isocalendar().week

In [14]:
# Zusätzlich wird noch eine Spalte für die Jahreszeit aus der zuvor generierten Monatsspalte erstellt 
train_df['Season'] = train_df['Month'].apply(lambda month: (month%12 // 3 + 1))

In [15]:
train_df['Temp'] = np.where(train_df['StateHoliday'] != 'Kein Feiertag', 1, 0)

# Erstellen Sie die Spalte "TagVorFeiertag", die 1 ist, wenn der nächste Tag ein Feiertag ist, und 0 sonst
train_df['TagVorFeiertag'] = train_df['Temp'].shift(-1).fillna(0)

# Erstellen Sie die Spalte "TagnachFeiertag", die 1 ist, wenn der vorherige Tag ein Feiertag war, und 0 sonst
train_df['TagnachFeiertag'] = train_df['Temp'].shift(1).fillna(0)
train_df.drop('Temp', axis=1, inplace=True)

In [16]:
train_df = pd.merge(train_df,store,how="inner",on="Store ID")
train_df.fillna(0,inplace=True)

# Exploratory Data Analysis

## Durchschnittliche Verkäufe

In [17]:
average_sales_per_day = train_df.groupby('Date')['Sales'].mean().reset_index()

fig = px.line(average_sales_per_day, y='Sales', x='Date', 
        labels={'Date': 'Kalendertag', 'Sales': 'Durchschnittlicher Umsatz'},
        title='Durchschnittlicher Umsatz pro Tag')
fig.show()

## Durchschnittliche Verkäufe pro Monat

In [18]:
average_sales_per_month = train_df.groupby('Month')['Sales'].mean().reset_index()

fig = px.bar(average_sales_per_month, x='Month', y='Sales',
             labels={'Month': 'Monat', 'Sales': 'Durchschnittlicher Umsatz'},
             title='Durchschnittlicher Umsatz pro Monat')
fig.update_xaxes(tickmode='array', tickvals=list(range(1, 13)),
                 ticktext=['Jan', 'Feb', 'Mär', 'Apr', 'Mai', 'Jun', 'Jul', 'Aug', 'Sep', 'Okt', 'Nov', 'Dez'])
fig.show()


- Der Umsatz variiert von Monat zu Monat, mit einigen Monaten, die höhere Durchschnittswerte aufweisen als andere.
- Es gibt keine extremen Spitzen oder Täler, aber es gibt eine sichtbare Variation im Bereich von etwa 5.570 € bis 6.883 € pro Monat im Durchschnitt.

## Durchschnittliche Verkäufe pro Woche

In [19]:
average_sales_per_week = train_df.groupby('WeekOfYear')['Sales'].mean().reset_index()

fig = px.line(average_sales_per_week, x='WeekOfYear', y='Sales',
              labels={'WeekOfYear': 'Kalenderwoche', 'Sales': 'Durchschnittlicher Umsatz'},
              title='Durchschnittlicher Umsatz pro Woche')
fig.update_xaxes(tickmode='array', tickvals=list(range(1, 54)))
fig.show()

- Der Umsatz variiert über das Jahr, mit einigen Spitzen und Tälern, was auf saisonale Einflüsse oder besondere Ereignisse hindeuten könnte. 
- Es gibt bestimmte Wochen, in denen der Umsatz deutlich höher ist als im Durchschnitt, was auf spezielle Verkaufsaktionen, Feiertage oder saisonale Ereignisse hinweisen könnte.

## Durchschnittliche Verkäufe pro Wochentag

In [20]:
average_sales_per_day = train_df.groupby('DayOfWeek')['Sales'].mean().reset_index()

fig = px.bar(average_sales_per_day, x='DayOfWeek', y='Sales',
             labels={'DayOfWeek': 'Wochentag', 'Sales': 'Durchschnittliche Verkäufe'},
             title='Durchschnittliche Verkäufe pro Wochentag')

fig.update_xaxes(tickmode='array', tickvals=[1, 2, 3, 4, 5, 6, 7],
                ticktext=['Montag', 'Dienstag', 'Mittwoch', 'Donnerstag', 'Freitag', 'Samstag', 'Sonntag'])
fig.show()

- Die Analyse der durchschnittlichen täglichen Verkäufe zeigt, dass der Beginn der Woche, insbesondere der Montag, für die Supermarktkette am umsatzstärksten ist, während die Verkaufszahlen gegen das Wochenende hin tendenziell abnehmen. 
- Auffallend ist der deutliche Einbruch der Verkaufszahlen am Sonntag, was darauf hindeutet, dass die meisten Filialen entweder geschlossen sind oder nur sehr begrenzt operieren.

# Aufgabe 1: Untersuchen Sie die Daten. Beantworten Sie dabei folgende Fragen bzw. Aufgaben

### 1. Wie viele Datenpunkte gibt es pro Store?

In [21]:
store_counts = train_df.groupby('Store ID').size().reset_index()
store_counts.columns = ['Store ID', 'Data Points']
unique_values = store_counts['Data Points'].value_counts().sort_index()

In [22]:
for i, v in unique_values.items():
    print(f"{v} Stores mit {i} Datenpunkten")


44 Stores mit 666 Datenpunkten
1 Stores mit 849 Datenpunkten
255 Stores mit 850 Datenpunkten


### 2. Visualisieren Sie die Verteilung der Verkaufszahlen für einige zufällig ausgewählte Stores und beschreiben Sie typische Muster, die Sie erkennen.

In [23]:
random_stores = np.random.choice(train_df['Store ID'].unique(), 5)
selected_stores_df = train_df[train_df['Store ID'].isin(random_stores)]

fig = px.histogram(selected_stores_df, x="Sales", color="Store ID", marginal="box", 
                   barmode="overlay", 
                   title="Verkaufszahlenverteilung für zufällig ausgewählte Stores",
                   height=600)

fig.update_xaxes(title_text='Verkaufszahlen')
fig.update_yaxes(title_text='Häufigkeit')
fig.show()

`Rechtsschiefe Verteilung`: Die meisten Filialen zeigen eine rechtsschiefe Verteilung der Verkaufszahlen. Das bedeutet, dass ein Großteil der Verkaufstage Verkaufszahlen im niedrigeren bis mittleren Bereich aufweist, während sehr hohe Verkaufszahlen seltener sind.

`Spitzen bei niedrigen Verkaufszahlen`: Es gibt Spitzen im Bereich sehr niedriger Verkaufszahlen. Dies könnte auf Tage hindeuten, an denen die Filialen geschlossen waren (wie Sonntage oder Feiertage) oder nur sehr begrenzt geöffnet hatten.

`Variation zwischen den Filialen`: Während einige Muster über die Filialen hinweg ähnlich sind, gibt es auch Unterschiede in den Verkaufszahlen, was auf verschiedene Standortfaktoren, Filialgrößen oder Kundenbasen hindeuten könnte.

### 3. Beschreiben Sie einige prägnante Zusammenhänge, die Ihnen zwischen verschiedenen Features auffallen.

In [24]:
train_df.groupby('DayOfWeek')['Sales'].mean()

DayOfWeek
1    7901.422431
2    7036.199486
3    6617.223371
4    6369.797242
5    6831.456849
6    6011.092986
7     125.497440
Name: Sales, dtype: float64

In [25]:
train_df.groupby('StoreType')['Sales'].mean()

StoreType
StoreType A     5878.099497
StoreType C     5938.888868
StoreType D     5640.378560
StoreTypee B    9348.329020
Name: Sales, dtype: float64

In [26]:
train_df.groupby('Assortment')['Sales'].mean()

Assortment
Basic       5518.713299
Extended    6161.353333
Extra       9609.298824
Name: Sales, dtype: float64

In [27]:
train_df['Sales'].corr(train_df['CompetitionDistance'])

-0.03567522080901384

In [28]:
train_df.groupby('SchoolHoliday')['Sales'].mean()

SchoolHoliday
0    5704.275184
1    6488.649432
Name: Sales, dtype: float64

In [29]:
train_df.groupby('Promo')['Sales'].mean()

Promo
0    4479.205735
1    8067.441795
Name: Sales, dtype: float64

In [30]:
train_df.corr(numeric_only=True)

,Store ID,DayOfWeek,Sales,Customers,Open,Promo,SchoolHoliday,Year,Month,Day,WeekOfYear,Season,TagVorFeiertag,TagnachFeiertag,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear
Store ID,1.000000e+00,-3.564871e-07,0.002610,0.019468,-0.001721,0.000040,-0.000181,0.000341,0.000842,0.000022,0.000819,0.000604,0.000968,0.001010,-0.055231,-0.016331,-0.039262,0.067460,0.003277,0.067364
DayOfWeek,-3.564871e-07,1.000000e+00,-0.447948,-0.389245,-0.534218,-0.393895,-0.211693,0.001060,-0.004684,0.002383,-0.002239,0.006950,0.028984,0.003222,-0.000029,0.000034,0.000025,0.000118,0.000168,0.000118
Sales,2.609588e-03,-4.479479e-01,1.000000,0.914052,0.659275,0.434767,0.075003,0.017092,0.043193,-0.011033,0.046544,-0.015127,0.053264,-0.011725,-0.035675,0.052101,0.062046,-0.139425,-0.070549,-0.139451
Customers,1.946830e-02,-3.892450e-01,0.914052,1.000000,0.611782,0.316520,0.066329,-0.001499,0.035291,-0.002391,0.038278,0.003999,0.047553,-0.019057,-0.109836,0.061680,0.088983,-0.189678,-0.121545,-0.189678
Open,-1.721330e-03,-5.342182e-01,0.659275,0.611782,1.000000,0.299531,0.078737,0.002016,-0.005740,0.035998,-0.001133,0.016709,0.035428,-0.055164,0.009735,0.007548,0.007890,-0.008748,-0.008479,-0.008750
Promo,3.958783e-05,-3.938946e-01,0.434767,0.316520,0.299531,1.000000,0.055701,0.025531,-0.013782,-0.105705,-0.000294,0.012611,-0.071866,-0.005381,0.000172,-0.000186,-0.000143,-0.000826,-0.001156,-0.000826
SchoolHoliday,-1.811101e-04,-2.116930e-01,0.075003,0.066329,0.078737,0.055701,1.000000,-0.036184,0.101233,0.015551,0.065138,0.048810,0.146992,0.111658,-0.001666,-0.001328,-0.003471,-0.006431,-0.007385,-0.006430
Year,3.413860e-04,1.059909e-03,0.017092,-0.001499,0.002016,0.025531,-0.036184,1.000000,-0.319445,-0.006145,-0.309757,-0.250976,-0.004432,-0.004172,0.001636,-0.001788,-0.001368,-0.007753,-0.010868,-0.007758
Month,8.419804e-04,-4.683789e-03,0.043193,0.035291,-0.005740,-0.013782,0.101233,-0.319445,1.000000,0.013083,0.967092,0.640187,0.007177,0.047201,0.004063,-0.004443,-0.003399,-0.019238,-0.026971,-0.019251
Day,2.179308e-05,2.383343e-03,-0.011033,-0.002391,0.035998,-0.105705,0.015551,-0.006145,0.013083,1.000000,0.064041,0.009000,-0.050028,0.090721,0.000061,-0.000062,-0.000050,-0.000318,-0.000442,-0.000318


In [31]:
exception_days = train_df[(train_df['DayOfWeek'] == 7) & (train_df['Open'] == 1) & (train_df['Sales'] > 0)]
px.line(exception_days, x='Date', y='Sales', color='Store ID')

- An Tagen mit Promotion (Promo = 1) liegt der durchschnittliche Umsatz bei etwa 8.067 €, während er an Tagen ohne Promotion (Promo = 0) bei etwa 4.479 € liegt. 
    - Promotionen hat einen deutlich positiven Einfluss auf den Umsatz haben.
- Der durchschnittliche Umsatz variiert je nach Wochentag, wobei Montag der umsatzstärkste Tag ist (ca. 7.901 €). Der Umsatz nimmt im Laufe der Woche tendenziell ab und erreicht am Sonntag den niedrigsten Wert (ca. 125 €).
    - Wochentag hat einen signifikanten Einfluss auf den Umsatz.
- Der durchschnittliche Umsatz während der Schulferien (SchoolHoliday = 1) liegt bei ca. 6.489 €, was höher ist als außerhalb der Schulferien (ca. 5.704 €). 
    - Dies könnte darauf hindeuten, dass Schulferien zu einem Anstieg des Umsatzes führen.
- Der durchschnittliche Umsatz variiert je nach Filialtyp als auch nach Sortimentskategorie. Filialen vom Typ B erzielen mit durchschnittlich etwa 9.348 € den höchsten Umsatz, gefolgt von Typ C, A und D.
    - Der Filialtyp als auch die Sortimentskategorie hat einen Einfluss auf den Umsatz haben.
- 

# Predictive Modeling

In [32]:
train_df['StoreType'] = train_df['StoreType'].map({"StoreType A": 1, "StoreTypee B": 2, "StoreType C": 3, "StoreType D": 4})
train_df['Assortment'] = train_df['Assortment'].map({"Basic": 0, "Extra": 1, "Extended": 2})
train_df['PromoInterval']= train_df['PromoInterval'].map({0:0, 'Jan,Apr,Jul,Oct' : 1, 'Feb,May,Aug,Nov' : 2, 'Mar,Jun,Sept,Dec' : 3})
train_df['StateHoliday'] = train_df['StateHoliday'].map({"Kein Feiertag": 0, "Feiertag": 1, "Ostern": 2, "Weihnachten": 3})

In [33]:
# train_df.drop(['Promo2SinceWeek', 'Promo2SinceYear', 'StateHoliday'], axis=1, inplace=True)

In [34]:
X = train_df
y = train_df['Sales']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [35]:
X_train

,Store ID,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,...,TagnachFeiertag,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
153571,188,3,2014-12-10,4258,404,1,0,0,0,2014,...,0.0,1,0,970.0,3.0,2013.0,1,31.0,2013.0,2
147077,180,7,2014-08-10,0,0,0,0,0,0,2014,...,0.0,1,2,15320.0,3.0,2011.0,1,14.0,2013.0,2
219272,266,6,2013-01-19,4508,437,1,0,0,0,2013,...,0.0,4,2,970.0,0.0,0.0,1,9.0,2011.0,3
174175,213,2,2014-03-11,3988,442,1,0,0,0,2014,...,0.0,4,2,3290.0,12.0,1999.0,1,35.0,2010.0,3
227019,276,4,2013-08-08,5244,577,1,0,0,1,2013,...,0.0,3,0,1600.0,8.0,2013.0,1,40.0,2014.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119879,146,3,2014-06-25,4287,535,1,0,0,0,2014,...,0.0,4,2,15720.0,3.0,2010.0,0,0.0,0.0,0
103694,127,5,2015-01-30,9820,673,1,1,0,0,2015,...,0.0,1,2,9070.0,4.0,2011.0,1,37.0,2009.0,1
131932,161,4,2013-11-14,4159,663,1,0,0,0,2013,...,0.0,1,0,620.0,9.0,2014.0,0,0.0,0.0,0
146867,180,7,2015-03-08,0,0,0,0,0,0,2015,...,0.0,1,2,15320.0,3.0,2011.0,1,14.0,2013.0,2


In [36]:
# spalten zum droppen
drop_colmn = ['Sales', 'Customers', 'Date']

# für StandardScaler
ss_colmn = ['Year', 'Month', 'Day', 'WeekOfYear', 'CompetitionDistance', 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear']

# für One Hot Encoding
ohe_colmn = ['StoreType', 'Assortment', 'PromoInterval']

# Aufgabe 4

## Vergleich

In [37]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import mean_squared_error, accuracy_score, r2_score
from time import time

In [38]:
models = [
    ('LinearRegression', LinearRegression()),
    ('Ridge', Ridge()),
    ('Lasso', Lasso()),
    ('DecisionTreeRegressor', DecisionTreeRegressor(random_state=42)),
    ('GradientBoostingRegressor', GradientBoostingRegressor(random_state=42)),
    ('RandomForestRegressor', RandomForestRegressor(random_state=42))
]

preprocessor = ColumnTransformer(
    transformers=[
        ('drop_cols', 'drop', drop_colmn),
        ('scale', StandardScaler(), ss_colmn),
        ('onehot', OneHotEncoder(handle_unknown='ignore'), ohe_colmn)
    ], remainder='passthrough')

In [39]:
results = []

for model_name, model in models:
    # Modellpipeline erstellen
    model_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])

    # Zeitmessung starten
    start_time = time()

    # Modell trainieren
    model_pipeline.fit(X_train, y_train)

    # Zeitmessung beenden
    end_time = time()
    training_time = end_time - start_time

    # Vorhersagen treffen
    y_pred = model_pipeline.predict(X_test)

    # Performance bewerten
    r2_score = model_pipeline.score(X_test, y_test)
    mse = mean_squared_error(y_test, y_pred)

    # Ergebnisse speichern
    results.append({
        'model_name': model_name,
        'r2_score': r2_score,
        'mse': mse,
        'training_time': training_time
    })

# Ergebnisse in DataFrame umwandeln
results_df = pd.DataFrame(results)

In [40]:
results_df

,model_name,r2_score,mse,training_time
0,LinearRegression,0.558415,7.109967e+06,0.417882
1,Ridge,0.558345,7.111084e+06,0.235370
2,Lasso,0.558313,7.111603e+06,2.092405
3,DecisionTreeRegressor,0.928202,1.156025e+06,1.860026
4,GradientBoostingRegressor,0.721038,4.491565e+06,41.124076
5,RandomForestRegressor,0.960353,6.383508e+05,120.533946


## Lineares Modell (Ridge)

In [41]:
# Vorverarbeitungspipeline erstellen
preprocessor = ColumnTransformer(
    transformers=[
        ('drop_cols', 'drop', drop_colmn),
        ('scale', StandardScaler(), ss_colmn),
        ('onehot', OneHotEncoder(handle_unknown='ignore'), ohe_colmn)
    ], remainder='passthrough')

# Modellpipeline erstellen
ridge_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', Ridge())
])

# Hyperparameter-Raster definieren
ridge_param_grid = {
    'regressor__alpha': [0.01, 0.1, 1, 10]
}

# Grid-Suche durchführen
tscv = TimeSeriesSplit(n_splits=5)
gs = GridSearchCV(ridge_model, param_grid=ridge_param_grid, cv=tscv, n_jobs=-1)
gs.fit(X_train, y_train)

GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None),
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('drop_cols',
                                                                         'drop',
                                                                         ['Sales',
                                                                          'Customers',
                                                                          'Date']),
                                                                        ('scale',
                                                                         StandardScaler(),
                                                                         ['Year',
                                                                          'Month',
                                                                          'Day',
                                                                          'WeekOfYear',
                                                                          'CompetitionDistance',
                                                                          'CompetitionOpenSinceMonth',
                                                                          'CompetitionOpenSinceYear']),
                                                                        ('onehot',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['StoreType',
                                                                          'Assortment',
                                                                          'PromoInterval'])])),
                                       ('regressor', Ridge())]),
             n_jobs=-1, param_grid={'regressor__alpha': [0.01, 0.1, 1, 10]})

In [42]:
y_pred_linear_regression = gs.predict(X_test)

In [43]:
print(f"Beste Hyperparameter-Kombination: {gs.best_params_}")
print(f"Accuracy Trainingsdaten: {gs.score(X_train, y_train)}")
print(f"Accuracy Testdaten: {gs.score(X_test, y_test)}")

Beste Hyperparameter-Kombination: {'regressor__alpha': 0.01}
Accuracy Trainingsdaten: 0.5610463288753407
Accuracy Testdaten: 0.5584128730160747


## Entscheidungsbaum

In [44]:
# Vorverarbeitungspipeline erstellen
preprocessor = ColumnTransformer(
    transformers=[
        ('drop_cols', 'drop', drop_colmn),
        ('scale', StandardScaler(), ss_colmn),
        ('onehot', OneHotEncoder(handle_unknown='ignore'), ohe_colmn)
    ], remainder='passthrough')

# Modellpipeline erstellen
tree_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', DecisionTreeRegressor(random_state=42))
])

# Hyperparameter-Raster definieren
tree_param_grid = {
    'regressor__max_depth': [20, 30, 40],
    'regressor__min_samples_split': [5, 10, 20],
    'regressor__min_samples_leaf': [1, 2, 4]
}

# Grid-Suche durchführen
tscv = TimeSeriesSplit(n_splits=5)
gs = GridSearchCV(tree_model, param_grid=tree_param_grid, cv=tscv, n_jobs=-1)
gs.fit(X_train, y_train)

GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None),
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('drop_cols',
                                                                         'drop',
                                                                         ['Sales',
                                                                          'Customers',
                                                                          'Date']),
                                                                        ('scale',
                                                                         StandardScaler(),
                                                                         ['Year',
                                                                          'Month',
                                                                          'Day',
                                                                          'WeekOfYear',
                                                                          'CompetitionDistance',
                                                                          'CompetitionOpenSinceMonth',
                                                                          'CompetitionOpenSinceYear']),
                                                                        ('onehot',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['StoreType',
                                                                          'Assortment',
                                                                          'PromoInterval'])])),
                                       ('regressor',
                                        DecisionTreeRegressor(random_state=42))]),
             n_jobs=-1,
             param_grid={'regressor__max_depth': [20, 30, 40],
                         'regressor__min_samples_leaf': [1, 2, 4],
                         'regressor__min_samples_split': [5, 10, 20]})

In [45]:
y_pred_entscheidungsbaum = gs.predict(X_test)

In [46]:
print(f"Beste Hyperparameter-Kombination: {gs.best_params_}")
print(f"Accuracy Trainingsdaten: {gs.score(X_train, y_train)}")
print(f"Accuracy Testdaten: {gs.score(X_test, y_test)}")

Beste Hyperparameter-Kombination: {'regressor__max_depth': 40, 'regressor__min_samples_leaf': 2, 'regressor__min_samples_split': 20}
Accuracy Trainingsdaten: 0.9694840257941157
Accuracy Testdaten: 0.9426498437782878


In [47]:
transformed_feature_names = gs.best_estimator_.named_steps['preprocessor'].get_feature_names_out()
importances = gs.best_estimator_.named_steps['regressor'].feature_importances_

# Create DataFrame
feature_importances = pd.DataFrame({
    'feature': transformed_feature_names,
    'importance': importances
})

feature_importances.sort_values(by='importance', ascending=False)

,feature,importance
20,remainder__Open,0.448162
4,scale__CompetitionDistance,0.157923
21,remainder__Promo,0.067185
18,remainder__Store ID,0.064790
5,scale__CompetitionOpenSinceMonth,0.040791
6,scale__CompetitionOpenSinceYear,0.032940
19,remainder__DayOfWeek,0.032281
29,remainder__Promo2SinceYear,0.027511
11,onehot__Assortment_0,0.015779
3,scale__WeekOfYear,0.015224


##  Ensemble-Modell (Randomforest)

In [48]:
# Vorverarbeitungspipeline erstellen
preprocessor = ColumnTransformer(
    transformers=[
        ('drop_cols', 'drop', drop_colmn),
        ('scale', StandardScaler(), ss_colmn),
        ('onehot', OneHotEncoder(handle_unknown='ignore'), ohe_colmn)
    ], remainder='passthrough')

# Modellpipeline erstellen
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_jobs=-1, random_state=42))
])

# Hyperparameter-Raster definieren
param_grid = {
    'regressor__n_estimators': [300], # [100, 200, 300],
    'regressor__max_depth': [40], # [20, 30, 40],
    #'regressor__max_features': [None, 'sqrt', 'log2'],
    'regressor__min_samples_split': [5], #[5, 10, 15],
}

# Grid-Suche durchführen
tscv = TimeSeriesSplit(n_splits=5)
gs = GridSearchCV(model, param_grid=param_grid, cv=tscv, n_jobs=-1)
gs.fit(X_train, y_train)

GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None),
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('drop_cols',
                                                                         'drop',
                                                                         ['Sales',
                                                                          'Customers',
                                                                          'Date']),
                                                                        ('scale',
                                                                         StandardScaler(),
                                                                         ['Year',
                                                                          'Month',
                                                                          'Day',
                                                                          'WeekOfYear',
                                                                          'CompetitionDistance',
                                                                          'CompetitionOpenSinceMonth',
                                                                          'CompetitionOpenSinceYear']),
                                                                        ('onehot',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['StoreType',
                                                                          'Assortment',
                                                                          'PromoInterval'])])),
                                       ('regressor',
                                        RandomForestRegressor(n_jobs=-1,
                                                              random_state=42))]),
             n_jobs=-1,
             param_grid={'regressor__max_depth': [40],
                         'regressor__min_samples_split': [5],
                         'regressor__n_estimators': [300]})

In [49]:
y_pred_random_forest = gs.predict(X_test)

In [50]:
print(f"Beste Hyperparameter-Kombination: {gs.best_params_}")
print(f"Accuracy Trainingsdaten: {gs.score(X_train, y_train)}")
print(f"Accuracy Testdaten: {gs.score(X_test, y_test)}")

Beste Hyperparameter-Kombination: {'regressor__max_depth': 40, 'regressor__min_samples_split': 5, 'regressor__n_estimators': 300}
Accuracy Trainingsdaten: 0.988849642023521
Accuracy Testdaten: 0.9604164933426717


In [51]:
transformed_feature_names = gs.best_estimator_.named_steps['preprocessor'].get_feature_names_out()
importances = gs.best_estimator_.named_steps['regressor'].feature_importances_

# Create DataFrame
feature_importances = pd.DataFrame({
    'feature': transformed_feature_names,
    'importance': importances
})

feature_importances.sort_values(by='importance', ascending=False)

,feature,importance
20,remainder__Open,0.438212
4,scale__CompetitionDistance,0.158121
21,remainder__Promo,0.065718
18,remainder__Store ID,0.064566
5,scale__CompetitionOpenSinceMonth,0.040170
19,remainder__DayOfWeek,0.033624
6,scale__CompetitionOpenSinceYear,0.032030
29,remainder__Promo2SinceYear,0.030693
3,scale__WeekOfYear,0.021001
2,scale__Day,0.018216


# Aufgabe 5

In [52]:
def calculate_total_cost(y_pred, y_test, model_name):
    # Kosten für Überschätzung
    overestimate_cost_1 = 150  # Kosten bei Überschätzung um mehr als 3000 EUR
    overestimate_cost_2 = 250  # Kosten bei Überschätzung um mehr als 6000 EUR

    # Kosten für Unterschätzung
    underestimate_cost = 100  # Kosten bei Unterschätzung um mehr als 4000 EUR

    # Berechne die Differenzen zwischen Vorhersagen und tatsächlichen Umsätzen
    diff = (y_pred - y_test).to_numpy()

    # Initialisiere Gesamtkosten
    total_cost = 0

    # Iteriere durch die Daten und berechne die Gesamtkosten
    for i in range(len(diff)):
        if diff[i] > 6000:
            total_cost += overestimate_cost_2
        elif diff[i] > 3000:
            total_cost += overestimate_cost_1
        elif diff[i] < -4000:
            total_cost += underestimate_cost

    # Drucke die Gesamtkosten für das Modell
    print(f"Gesamtkosten {model_name}:", total_cost)

# Now you can call this function for each model
calculate_total_cost(y_pred_linear_regression, y_test, "Linear Regression")
calculate_total_cost(y_pred_entscheidungsbaum, y_test, "Entscheidungsbaum")
calculate_total_cost(y_pred_random_forest, y_test, "Random Forest")

Gesamtkosten Linear Regression: 1036400
Gesamtkosten Entscheidungsbaum: 96750
Gesamtkosten Random Forest: 50900
